In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import requests 
import time 
df=pd.read_csv("Levels_Fyi_Salary_Data.csv", delimiter=",")
df.info()
df.head(60)

In [ ]:
#Connor
# take the location column and seperate state and city and index them, take min/max of base salary/totalyearlycompensation 

In [ ]:
#take base salary/totalyearlycompensation 

In [ ]:
#group by company and min/ max yearly compensation

In [ ]:
#Education compared to salary/totalyearlycompensation 

In [ ]:
#Alfredo
#years of experience/ years at company and comparison to yearly compensation

In [ ]:
#Neel
#male/ female convert to usable data and compare to yearly compensation